# Query Understanding — Intent Classification + Slot Filling (C# / .NET Interactive)

This notebook is a hands-on sandbox for building a **rule‑first** intent classification and slot-filling pipeline using:
- `Microsoft.Recognizers.Text` for date/time parsing
- `FuzzySharp` for fuzzy name matching
- Plain C# for domain maps and query building

> Open this in **.NET Interactive** (C#) kernel.


### Intents

**Intent Classification:** What does  the user want? e.g.,
- get contact info
- filter by hire date
- filter by role

One way to classify intents is by using regular expressions

### Slots

**Slot Filling:** Slots are the key pieces of information needed to fulfill the intent. e.g.,
- names
- dates
- roles
- departments

Various ways to extract slots using regex, fuzzy matching, and Microsoft Recognizers for date/time.

### Domain Mapping
Maps are used to match synonyms (like "team lead" or "eng") to canonical role/department names.

### Pipeline Steps
- Normalize the query (lowercase, trim, etc.)
- Classify the intent
- Extract slots (names, dates, roles, departments)
- Apply filters to the employee list based on the extracted info


In [2]:
// NuGet references (only needed in the notebook environment)
#r "nuget: FuzzySharp, 2.0.2"
#r "nuget: Microsoft.Recognizers.Text, 1.8.13"
#r "nuget: Microsoft.Recognizers.Text.DateTime, 1.8.13"

using System;
using System.Collections.Generic;
using System.Globalization;
using System.Linq;
using System.Text.RegularExpressions;
using FuzzySharp;
using Microsoft.Recognizers.Text;
using Microsoft.Recognizers.Text.DateTime;


Installed Packages FuzzySharp, 2.0.2 Microsoft.Recognizers.Text, 1.8.13 Microsoft.Recognizers.Text.DateTime, 1.8.13

In [3]:
public enum Intent { GetContactInfo, FilterByHireDate, FilterByRole, Unknown }

public record Slots(
    string[]? Names = null,
    DateTime? Date = null,
    (DateTime Start, DateTime End)? Range = null,
    string? Operator = null,
    string? Department = null,
    string? Role = null
);

public record QuerySpec(Intent Intent, Slots Slots);

public record Employee(
    string DisplayName,
    string Email,
    string Department,
    string Role,
    DateTime OriginalHireDate
);


In [4]:
public sealed class RoleMap
{
    private readonly (string Canonical, string[] Synonyms)[] _map = new[]
    {
        ("Manager", new[] { "manager", "managers", "supervisor", "team lead", "lead" }),
        ("Engineer", new[] { "engineer", "staff engineer", "developer" }),
        ("PM", new[] { "pm", "product manager" }),
        ("Director", new[] { "director" }),
    };

    public string? ToCanonicalRole(string normalized)
    {
        foreach (var (canon, syns) in _map)
            if (syns.Any(s => Regex.IsMatch(normalized, $@"\b{Regex.Escape(s)}\b")))
                return canon;
        return null;
    }
}

public sealed class DepartmentMap
{
    private readonly (string Canonical, string[] Synonyms)[] _map = new[]
    {
        ("Engineering", new[] { "engineering", "eng", "dev" }),
        ("Product",     new[] { "product", "pm team" }),
        ("HR",          new[] { "hr", "human resources" }),
        ("Sales",       new[] { "sales", "revenue" })
    };

    public string? ToCanonicalDepartment(string normalized)
    {
        foreach (var (canon, syns) in _map)
            if (syns.Any(s => Regex.IsMatch(normalized, $@"\b{Regex.Escape(s)}\b")))
                return canon;
        return null;
    }
}


In [5]:
public static class QueryUnderstanding
{
    // 1) Normalize input
    public static string Normalize(string input) =>
        input.Trim().ToLowerInvariant().Replace("’", "'");

    // 2) Intent classifier (rule-first)
    public static Intent ClassifyIntent(string normalized)
    {
        if (Regex.IsMatch(normalized, @"\b(email|contact|mail)\b")) return Intent.GetContactInfo;

        if (Regex.IsMatch(normalized, @"\b(hire|start|joined|hired|tenure|employment)\b") &&
            Regex.IsMatch(normalized, @"\b(before|after|between|on|since|last year|in \d{4})\b"))
            return Intent.FilterByHireDate;

        if (Regex.IsMatch(normalized, @"\b(manager|managers|lead|leads|supervisor|director|vp|cto|staff engineer|engineer|pm|product manager|designer)\b"))
            return Intent.FilterByRole;

        return Intent.Unknown;
    }

    // 3) Slot filling
    public static Slots FillSlots(
        string normalized,
        IReadOnlyList<Employee> employees,
        RoleMap roleMap,
        DepartmentMap deptMap)
    {
        var names = ExtractNames(normalized, employees);
        var (date, range, op) = ExtractDateInfo(normalized);
        var role = ExtractRole(normalized, roleMap);
        var department = ExtractDepartment(normalized, deptMap);

        op ??= range.HasValue ? "between"
             : Regex.IsMatch(normalized, @"\bbefore\b") ? "before"
             : Regex.IsMatch(normalized, @"\bafter\b") ? "after"
             : Regex.IsMatch(normalized, @"\bon\b|\bin\b") ? "on"
             : null;

        return new Slots(
            Names: names.Length > 0 ? names : null,
            Date: date,
            Range: range,
            Operator: op,
            Department: department,
            Role: role
        );
    }

    private static string[] ExtractNames(string normalized, IReadOnlyList<Employee> employees)
    {
        var tokens = Regex.Matches(normalized, @"[a-z']+").Select(m => m.Value).ToArray();
        if (tokens.Length == 0) return Array.Empty<string>();

        var unique = new HashSet<string>(StringComparer.OrdinalIgnoreCase);
        foreach (var token in tokens)
        {
            if (token.Length < 2 || token is "email" or "and" or "the" or "managers") continue;

            var best = employees
                .Select(e => new
                {
                    Name = e.DisplayName,
                    Score = Math.Max(
                        Fuzz.PartialRatio(token, e.DisplayName.Split(' ')[0]),
                        Fuzz.PartialRatio(token, e.DisplayName.Split(' ').Last()))
                })
                .OrderByDescending(x => x.Score)
                .FirstOrDefault();

            if (best is { Score: >= 85 }) unique.Add(best.Name);
        }
        return unique.ToArray();
    }

    private static (DateTime? Date, (DateTime Start, DateTime End)? Range, string? Operator)
        ExtractDateInfo(string input)
    {
        var culture = Culture.English;
        var results = DateTimeRecognizer.RecognizeDateTime(input, culture);

        DateTime? single = null;
        (DateTime Start, DateTime End)? range = null;
        string? op = null;

        foreach (var r in results)
        {
            if (!r.Resolution.TryGetValue("values", out var valuesObj) ||
                valuesObj is not List<Dictionary<string, string>> values) continue;

            var first = values.FirstOrDefault();
            if (first is null) continue;

            if (first.TryGetValue("type", out var t))
            {
                if (t.Contains("dateRange") && first.TryGetValue("start", out var s) && first.TryGetValue("end", out var e))
                {
                    if (DateTime.TryParse(s, CultureInfo.InvariantCulture, DateTimeStyles.AssumeUniversal, out var start) &&
                        DateTime.TryParse(e, CultureInfo.InvariantCulture, DateTimeStyles.AssumeUniversal, out var end))
                    {
                        range = (start.Date, end.Date);
                        op ??= Regex.IsMatch(input, @"\bbetween\b", RegexOptions.IgnoreCase) ? "between" : "range";
                    }
                }
                else if (t is "date" && first.TryGetValue("value", out var v) &&
                         DateTime.TryParse(v, CultureInfo.InvariantCulture, DateTimeStyles.AssumeUniversal, out var d))
                {
                    single = d.Date;
                }
            }
        }

        if (single is null && range is null)
        {
            var yearMatch = Regex.Match(input, @"\b(19|20)\d{2}\b");
            if (yearMatch.Success && int.TryParse(yearMatch.Value, out var y))
            {
                var cutoff = new DateTime(y, 1, 1, 0, 0, 0, DateTimeKind.Utc);
                if (Regex.IsMatch(input, @"\bbefore\b", RegexOptions.IgnoreCase)) { op = "before"; single = cutoff; }
                else if (Regex.IsMatch(input, @"\bafter\b", RegexOptions.IgnoreCase)) { op = "after"; single = cutoff; }
                else if (Regex.IsMatch(input, @"\bon\b|\bin\b", RegexOptions.IgnoreCase)) { op = "on"; single = cutoff; }
            }
            else if (Regex.IsMatch(input, @"\blast year\b", RegexOptions.IgnoreCase))
            {
                var now = DateTime.UtcNow;
                range = (new DateTime(now.Year - 1, 1, 1, 0, 0, 0, DateTimeKind.Utc),
                         new DateTime(now.Year - 1, 12, 31, 0, 0, 0, DateTimeKind.Utc));
                op = "between";
            }
        }

        return (single, range, op);
    }

    private static string? ExtractRole(string normalized, RoleMap map) => map.ToCanonicalRole(normalized);
    private static string? ExtractDepartment(string normalized, DepartmentMap map) => map.ToCanonicalDepartment(normalized);

    // 4) Apply filters
    public static IEnumerable<Employee> ApplyFilters(IEnumerable<Employee> employees, QuerySpec spec)
    {
        var q = employees;

        if (spec.Slots.Names is { Length: > 0 })
            q = q.Where(e => spec.Slots.Names!.Contains(e.DisplayName, StringComparer.OrdinalIgnoreCase));

        if (!string.IsNullOrWhiteSpace(spec.Slots.Department))
            q = q.Where(e => string.Equals(e.Department, spec.Slots.Department, StringComparison.OrdinalIgnoreCase));

        if (!string.IsNullOrWhiteSpace(spec.Slots.Role))
            q = q.Where(e => e.Role.Contains(spec.Slots.Role!, StringComparison.OrdinalIgnoreCase));

        if (spec.Intent == Intent.FilterByHireDate || spec.Slots.Operator is not null)
        {
            var op = spec.Slots.Operator;
            if (spec.Slots.Range is { } r)
                q = q.Where(e => e.OriginalHireDate.Date >= r.Start.Date && e.OriginalHireDate.Date <= r.End.Date);
            else if (spec.Slots.Date is { } d)
            {
                q = op switch
                {
                    "before" => q.Where(e => e.OriginalHireDate.Date < d.Date),
                    "after"  => q.Where(e => e.OriginalHireDate.Date > d.Date),
                    "on"     => q.Where(e => e.OriginalHireDate.Date == d.Date),
                    _        => q
                };
            }
        }

        return q;
    }
}


In [6]:
var employees = new List<Employee> {
    new("Rick Sanchez",   "rick.sanchez@company.com",   "Engineering", "Staff Engineer",  new DateTime(2015,  5, 10)),
    new("Morty Smith",    "morty.smith@company.com",    "Engineering", "Engineer I",      new DateTime(2023, 10, 12)),
    new("Summer Smith",   "summer.smith@company.com",   "Product",     "PM",              new DateTime(2021,  2,  1)),
    new("Beth Smith",     "beth.smith@company.com",     "HR",          "HR Manager",      new DateTime(2019,  7,  3)),
    new("Jerry Smith",    "jerry.smith@company.com",    "Sales",       "Account Manager", new DateTime(2022,  9, 15))
};

Console.WriteLine($"Loaded demo employees: {employees.Count}");


Loaded demo employees: 5


In [7]:
string Ask(string query)
{
    var text = QueryUnderstanding.Normalize(query);
    var intent = QueryUnderstanding.ClassifyIntent(text);
    var slots = QueryUnderstanding.FillSlots(text, employees, new RoleMap(), new DepartmentMap());
    var spec = new QuerySpec(intent, slots);
    var results = QueryUnderstanding.ApplyFilters(employees, spec).ToList();

    string header = $"Intent: {intent} | Slots: " +
                    $"names=[{string.Join(", ", slots.Names ?? Array.Empty<string>())}] " +
                    $"dept={slots.Department ?? "∅"} role={slots.Role ?? "∅"} " +
                    $"op={slots.Operator ?? "∅"} " +
                    (slots.Range is { } r ? $"range=({r.Start:yyyy-MM-dd}..{r.End:yyyy-MM-dd})" :
                     slots.Date is { } d ? $"date={d:yyyy-MM-dd}" : "date=∅");

    var body = intent switch
    {
        Intent.GetContactInfo    => string.Join('\n', results.Select(e => $"{e.DisplayName}: {e.Email}")),
        Intent.FilterByRole      => string.Join('\n', results.Select(e => $"{e.DisplayName} — {e.Role}")),
        Intent.FilterByHireDate  => string.Join('\n', results.Select(e => $"{e.DisplayName} — {e.OriginalHireDate:yyyy-MM-dd}")),
        _                        => results.Count == 0 ? "No matches." : string.Join('\n', results.Select(e => e.DisplayName))
    };

    return header + "\n" + body;
}

// Try a few:
Console.WriteLine(Ask("what are rick, summer, and morty's emails?"));
Console.WriteLine();
Console.WriteLine(Ask("show managers in engineering hired before 2024"));
Console.WriteLine();
Console.WriteLine(Ask("who was hired between 2020 and 2023?"));


Intent: Unknown | Slots: names=[] dept=∅ role=∅ op=∅ date=∅
Rick Sanchez
Morty Smith
Summer Smith
Beth Smith
Jerry Smith

Intent: FilterByHireDate | Slots: names=[] dept=Engineering role=Manager op=before date=2024-01-01


Intent: FilterByHireDate | Slots: names=[] dept=∅ role=∅ op=∅ date=∅
Rick Sanchez — 2015-05-10
Morty Smith — 2023-10-12
Summer Smith — 2021-02-01
Beth Smith — 2019-07-03
Jerry Smith — 2022-09-15


## Next steps

- Expand `RoleMap` / `DepartmentMap` with your org-specific synonyms and buckets.
- Replace the in-memory list with your data source, keeping the **ApplyFilters** logic pure.
- Add unit tests in a separate project (xUnit) that assert `ClassifyIntent`, `FillSlots`, and `ApplyFilters` behavior.
- When you're ready, train a tiny ML.NET multiclass model for `Intent` and keep slot filling rule-based.
